In [15]:
%%capture
!python -m textblob.download_corpora

In [16]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import graphviz
from IPython.display import display
from sklearn.model_selection import GridSearchCV
import pickle
from textblob import TextBlob
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
from textblob import download_corpora
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('omw-1.4')
import re

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/joeyrobak/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/joeyrobak/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/joeyrobak/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:
path = '/Users/joeyrobak/NM_UFO_DATA_ALL.xlsx'
df = pd.read_excel(path)
df

,Link,Occurred,City,State,Country,Shape,Summary,Reported,Media,Explanation
0,Open,2024-09-30 18:45:00,Tres Piedras,NM,USA,Other,White hovering figure 8,2024-10-01,NaN,NaN
1,Open,2024-09-20 19:30:00,Soham,NM,USA,Triangle,Bright light moving in erratic pattern over Ro...,2024-09-25,Y,NaN
2,Open,2024-09-11 19:25:00,Questa,NM,USA,Other,Observed a figure 8 shaped object as bright as...,2024-09-11,NaN,NaN
3,Open,2024-08-18 04:00:00,Albuquerque,NM,USA,Other,I saw two grays in my living room.,2024-08-29,NaN,NaN
4,Open,2024-08-07 20:54:00,Rio Rancho,NM,USA,Diamond,Smaller light gets absorbed by larger light an...,2024-08-08,Y,NaN
...,...,...,...,...,...,...,...,...,...,...
1698,Open,1945-08-16 11:30:00,San Antonio,NM,USA,Oval,THE CRAFT APPEARED TO HAVE DECENDED AT AN ANGL...,2003-11-10,NaN,NaN
1699,Open,1944-06-01 12:00:00,Los Alamos,NM,USA,Disk,Disk hovers over Los Alamos during the develop...,2009-05-03,NaN,NaN
1700,Open,1944-04-02 11:00:00,Clovis,NM,USA,Disk,"Saw UFO in 1944 over an airbase in New Mexico,...",2005-03-17,NaN,NaN
1701,Open,1929-01-28 15:41:00,Santa Teresa,NM,USA,Circle,large craft with other smaller crafts moving f...,2021-01-06,NaN,NaN


In [18]:
comments = df['Summary']


In [19]:
comments

0                                 White hovering figure 8
1       Bright light moving in erratic pattern over Ro...
2       Observed a figure 8 shaped object as bright as...
3                      I saw two grays in my living room.
4       Smaller light gets absorbed by larger light an...
                              ...                        
1698    THE CRAFT APPEARED TO HAVE DECENDED AT AN ANGL...
1699    Disk hovers over Los Alamos during the develop...
1700    Saw UFO in 1944 over an airbase in New Mexico,...
1701    large craft with other smaller crafts moving f...
1702    object was bright, not natural, IT was not a p...
Name: Summary, Length: 1703, dtype: object

In [20]:
comments.value_counts()

Summary
MADAR Node 78                                                                                                                              9
Driving I10 west of Las Cruces NM a random shape object visable for more than 60 mins seem to follow my U-Hall truck.                      2
2 witnesses spotted several orange and white lights bobbing and flying erratically in the sky. Specifically 2 orange flying objects tha    2
steadily moved across the sky & then hovered, 2x, creating a triangle formation & then dissapearing.                                       2
MADAR Node 23                                                                                                                              2
                                                                                                                                          ..
Orange orbs fly over neighborhood.                                                                                                         1
Girlf

In [21]:
full_list = comments[:]
converted_data = [str(item) if isinstance(item, float) else item for item in full_list]

print(converted_data)

['White hovering figure 8', 'Bright light moving in erratic pattern over Rowe Mesa', 'Observed a figure 8 shaped object as bright as the moon with binoculars and naked eye for 25 minutes in Northen New Mexico 7:25 MST.', 'I saw two grays in my living room.', 'Smaller light gets absorbed by larger light and then just disappears.', 'White light arcs across sky and returns in same direction', 'Fast moving black orb aircraft', 'East security camera at about 00:00 hrs', 'Object moved faster than a commercial jet , no sound from south to north at a constant speed .', 'White orb hovering over mountain.', '2 triangular (like a paper airplane) flying objects', '3 “stars” seeming to move in relation to one another and forming a moving triangle', '9 orange orbs in 3 groups', 'Black rectangular craft with red lights breaks sound barrier with no sound.', 'Large bright white light', "Unexplained lights that don't move like aircraft", 'Flash bulb-like lights against a clear, dark sky.', '3 star-looki

In [22]:
sc = comments[0:50]
sc

0                               White hovering figure 8
1     Bright light moving in erratic pattern over Ro...
2     Observed a figure 8 shaped object as bright as...
3                    I saw two grays in my living room.
4     Smaller light gets absorbed by larger light an...
5     White light arcs across sky and returns in sam...
6                        Fast moving black orb aircraft
7               East security camera at about 00:00 hrs
8     Object moved faster than a commercial jet , no...
9                     White orb hovering over mountain.
10    2 triangular (like a paper airplane) flying ob...
11    3 “stars” seeming to move in relation to one a...
12                            9 orange orbs in 3 groups
13    Black rectangular craft with red lights breaks...
14                             Large bright white light
15     Unexplained lights that don't move like aircraft
16    Flash bulb-like lights against a clear, dark sky.
17    3 star-looking lights falling down toward 

In [23]:
sentiment = sc.apply(lambda x: TextBlob(x).sentiment)


In [24]:
sentiment.sort_values()

20    (-0.30000000000000004, 0.39999999999999997)
43                                   (-0.1, 0.15)
36    (-0.07777777777777778, 0.14444444444444446)
17    (-0.07777777777777778, 0.19444444444444448)
8     (-0.06666666666666667, 0.24444444444444446)
15                                   (-0.05, 0.0)
32                   (-0.03392857142857143, 0.25)
16                  (-0.025, 0.39166666666666666)
0                                      (0.0, 0.0)
12                                     (0.0, 0.0)
11                                     (0.0, 0.0)
9                                      (0.0, 0.0)
7                                      (0.0, 0.0)
31                                     (0.0, 0.0)
23                                     (0.0, 0.0)
3                                      (0.0, 0.0)
40                                     (0.0, 0.0)
10                                     (0.0, 0.0)
18                                     (0.0, 0.0)
27                     (0.0, 0.05833333333333333)


In [25]:
tokenized_data = []

# Loop through each row in the data series
for text in sc:
    # Convert the text to a TextBlob
    blob = TextBlob(text)
    # Tokenize the text by words
    tokens = blob.words
    # Append the tokens to the new list
    tokenized_data.append(tokens)

# Convert the list of tokens back into a pandas Series if needed
tokenized_series = pd.Series(tokenized_data)
print(tokenized_series)

0                          [White, hovering, figure, 8]
1     [Bright, light, moving, in, erratic, pattern, ...
2     [Observed, a, figure, 8, shaped, object, as, b...
3            [I, saw, two, grays, in, my, living, room]
4     [Smaller, light, gets, absorbed, by, larger, l...
5     [White, light, arcs, across, sky, and, returns...
6                  [Fast, moving, black, orb, aircraft]
7       [East, security, camera, at, about, 00:00, hrs]
8     [Object, moved, faster, than, a, commercial, j...
9                [White, orb, hovering, over, mountain]
10    [2, triangular, like, a, paper, airplane, flyi...
11    [3, “, stars, ”, seeming, to, move, in, relati...
12                     [9, orange, orbs, in, 3, groups]
13    [Black, rectangular, craft, with, red, lights,...
14                        [Large, bright, white, light]
15    [Unexplained, lights, that, do, n't, move, lik...
16    [Flash, bulb-like, lights, against, a, clear, ...
17    [3, star-looking, lights, falling, down, t

In [26]:
sc[30]

"One Video it's Coming to my house. Next video it's there for several seconds LOUD BUZZING . 3rd video GONE Like Nothing."

In [27]:
sentence = tokenized_series

In [28]:
tokenized_data2 = []

# Loop through each row in the data series
for text in sentence:
    # Convert the text to a TextBlob
    # Tokenize the text by words
   # count = sorted(text.word_counts.items())
    count = Counter(text)
    # Append the tokens to the new list
    tokenized_data2.append(count)

# Convert the list of tokens back into a pandas Series if needed
tokenized_series2 = pd.Series(tokenized_data2)
print(tokenized_series2)

0      {'White': 1, 'hovering': 1, 'figure': 1, '8': 1}
1     {'Bright': 1, 'light': 1, 'moving': 1, 'in': 1...
2     {'Observed': 1, 'a': 1, 'figure': 1, '8': 1, '...
3     {'I': 1, 'saw': 1, 'two': 1, 'grays': 1, 'in':...
4     {'Smaller': 1, 'light': 2, 'gets': 1, 'absorbe...
5     {'White': 1, 'light': 1, 'arcs': 1, 'across': ...
6     {'Fast': 1, 'moving': 1, 'black': 1, 'orb': 1,...
7     {'East': 1, 'security': 1, 'camera': 1, 'at': ...
8     {'Object': 1, 'moved': 1, 'faster': 1, 'than':...
9     {'White': 1, 'orb': 1, 'hovering': 1, 'over': ...
10    {'2': 1, 'triangular': 1, 'like': 1, 'a': 1, '...
11    {'3': 1, '“': 1, 'stars': 1, '”': 1, 'seeming'...
12    {'9': 1, 'orange': 1, 'orbs': 1, 'in': 1, '3':...
13    {'Black': 1, 'rectangular': 1, 'craft': 1, 'wi...
14    {'Large': 1, 'bright': 1, 'white': 1, 'light': 1}
15    {'Unexplained': 1, 'lights': 1, 'that': 1, 'do...
16    {'Flash': 1, 'bulb-like': 1, 'lights': 1, 'aga...
17    {'3': 1, 'star-looking': 1, 'lights': 1, '

In [29]:
import re
from collections import Counter

# Assuming full_list is defined somewhere in your code
# full_list = [...]

# Convert all items in full_list to strings
full_list = [str(item) for item in full_list]

# Step 1: Combine all strings in the list into a single string
all_text = " ".join(full_list)

# Step 2: Tokenize words (splitting by non-word characters to handle punctuation, etc.)
words = re.findall(r'\b\w+\b', all_text.lower())  # Convert to lowercase for uniformity

# Define stop words
stop_words = {
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves",
    "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was",
    "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and",
    "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between",
    "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off",
    "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any",
    "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",
    "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"
}

# Step 3: Filter out stop words
filtered_words = [word for word in words if word not in stop_words]

# Step 4: Count frequency of each word
word_counts = Counter(filtered_words)

# Step 5: Convert counts to a list of tuples (or any other format you prefer)
# This will give you a list like [('apple', 5), ('banana', 3), ('orange', 1)]
word_count_list = list(word_counts.items())

# Step 6: Sort the list of tuples by the frequency of each word in descending order
sorted_word_count_list = sorted(word_count_list, key=lambda x: x[1], reverse=True)

# Print the sorted list of word counts
print(sorted_word_count_list)


[('light', 404), ('lights', 353), ('bright', 321), ('sky', 316), ('object', 311), ('white', 183), ('moving', 172), ('saw', 165), ('nm', 165), ('new', 139), ('mexico', 134), ('red', 134), ('shaped', 133), ('orange', 119), ('large', 118), ('like', 116), ('craft', 114), ('albuquerque', 110), ('flying', 102), ('seen', 99), ('ufo', 98), ('green', 96), ('nuforc', 89), ('pd', 89), ('two', 88), ('note', 88), ('2', 86), ('shape', 82), ('west', 79), ('objects', 77), ('south', 74), ('hovering', 72), ('across', 72), ('one', 71), ('blue', 71), ('night', 70), ('high', 70), ('fast', 67), ('star', 67), ('east', 64), ('3', 64), ('flashing', 62), ('north', 60), ('triangle', 59), ('speed', 56), ('looked', 56), ('formation', 56), ('driving', 56), ('santa', 54), ('appeared', 52), ('three', 52), ('around', 51), ('traveling', 50), ('near', 50), ('fe', 49), ('report', 49), ('noticed', 48), ('orb', 47), ('low', 47), ('glowing', 44), ('disappeared', 44), ('moved', 43), ('1', 43), ('sound', 42), ('observed', 41)

In [30]:
    
all_text = (" ".join(full_list))

# Step 2: Tokenize words (splitting by non-word characters to handle punctuation, etc.)
words = re.findall(r'\b\w+\b', all_text.lower())  # Convert to lowercase for uniformity

# Step 3: Count frequency of each word
word_counts = Counter(words)

# Step 4: Convert counts to a list of tuples (or any other format you prefer)
# This will give you a list like [('apple', 5), ('banana', 3), ('orange', 1)]
word_count_list = list(word_counts.items())
   
sorted_word_count_list = sorted(word_count_list, key=lambda x: x[1], reverse=True)

print(sorted_word_count_list)


[('the', 793), ('a', 709), ('in', 704), ('and', 555), ('of', 536), ('light', 404), ('i', 396), ('to', 375), ('lights', 353), ('bright', 321), ('sky', 316), ('object', 311), ('was', 294), ('over', 280), ('at', 231), ('it', 226), ('on', 199), ('with', 195), ('white', 183), ('moving', 172), ('saw', 165), ('nm', 165), ('from', 164), ('my', 154), ('then', 151), ('new', 139), ('that', 135), ('mexico', 134), ('red', 134), ('shaped', 133), ('orange', 119), ('large', 118), ('like', 116), ('craft', 114), ('albuquerque', 110), ('very', 109), ('for', 105), ('flying', 102), ('we', 99), ('seen', 99), ('ufo', 98), ('green', 96), ('an', 94), ('about', 93), ('no', 89), ('out', 89), ('nuforc', 89), ('pd', 89), ('two', 88), ('were', 88), ('note', 88), ('2', 86), ('by', 84), ('shape', 82), ('west', 79), ('objects', 77), ('above', 76), ('south', 74), ('up', 74), ('hovering', 72), ('as', 72), ('across', 72), ('one', 71), ('blue', 71), ('night', 70), ('high', 70), ('fast', 67), ('star', 67), ('or', 66), ('ea

In [31]:
stop_words = {
    "i": True,
    "me": True,
    "my": True,
    "myself": True,
    "we": True,
    "our": True,
    "ours": True,
    "ourselves": True,
    "you": True,
    "your": True,
    "yours": True,
    "yourself": True,
    "yourselves": True,
    "he": True,
    "him": True,
    "his": True,
    "himself": True,
    "she": True,
    "her": True,
    "hers": True,
    "herself": True,
    "it": True,
    "its": True,
    "itself": True,
    "they": True,
    "them": True,
    "their": True,
    "theirs": True,
    "themselves": True,
    "what": True,
    "which": True,
    "who": True,
    "whom": True,
    "this": True,
    "that": True,
    "these": True,
    "those": True,
    "am": True,
    "is": True,
    "are": True,
    "was": True,
    "were": True,
    "be": True,
    "been": True,
    "being": True,
    "have": True,
    "has": True,
    "had": True,
    "having": True,
    "do": True,
    "does": True,
    "did": True,
    "doing": True,
    "a": True,
    "an": True,
    "the": True,
    "and": True,
    "but": True,
    "if": True,
    "or": True,
    "because": True,
    "as": True,
    "until": True,
    "while": True,
    "of": True,
    "at": True,
    "by": True,
    "for": True,
    "with": True,
    "about": True,
    "against": True,
    "between": True,
    "into": True,
    "through": True,
    "during": True,
    "before": True,
    "after": True,
    "above": True,
    "below": True,
    "to": True,
    "from": True,
    "up": True,
    "down": True,
    "in": True,
    "out": True,
    "on": True,
    "off": True,
    "over": True,
    "under": True,
    "again": True,
    "further": True,
    "then": True,
    "once": True,
    "here": True,
    "there": True,
    "when": True,
    "where": True,
    "why": True,
    "how": True,
    "all": True,
    "any": True,
    "both": True,
    "each": True,
    "few": True,
    "more": True,
    "most": True,
    "other": True,
    "some": True,
    "such": True,
    "no": True,
    "nor": True,
    "not": True,
    "only": True,
    "own": True,
    "same": True,
    "so": True,
    "than": True,
    "too": True,
    "very": True,
    "s": True,
    "t": True,
    "can": True,
    "will": True,
    "just": True,
    "don": True,
    "should": True,
    "now": True
}

In [32]:
filtered_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

print(filtered_counts)


{'white': 183, 'hovering': 72, 'figure': 5, '8': 19, 'bright': 321, 'light': 404, 'moving': 172, 'erratic': 4, 'pattern': 10, 'rowe': 1, 'mesa': 11, 'observed': 41, 'shaped': 133, 'object': 311, 'moon': 36, 'binoculars': 5, 'naked': 2, 'eye': 5, '25': 17, 'minutes': 33, 'northen': 1, 'new': 139, 'mexico': 134, '7': 22, 'mst': 2, 'saw': 165, 'two': 88, 'grays': 1, 'living': 4, 'room': 7, 'smaller': 13, 'gets': 2, 'absorbed': 1, 'larger': 9, 'disappears': 18, 'arcs': 1, 'across': 72, 'sky': 316, 'returns': 1, 'direction': 16, 'fast': 67, 'black': 29, 'orb': 47, 'aircraft': 36, 'east': 64, 'security': 3, 'camera': 13, '00': 13, 'hrs': 5, 'moved': 43, 'faster': 4, 'commercial': 6, 'jet': 15, 'sound': 42, 'south': 74, 'north': 60, 'constant': 3, 'speed': 56, 'mountain': 27, '2': 86, 'triangular': 38, 'like': 116, 'paper': 3, 'airplane': 12, 'flying': 102, 'objects': 77, '3': 64, 'stars': 23, 'seeming': 1, 'move': 28, 'relation': 1, 'one': 71, 'another': 13, 'forming': 4, 'triangle': 59, '9'

In [33]:
print(word_count_list.num_sort())

AttributeError: 'list' object has no attribute 'num_sort'

In [ ]:
sorted(text.word_counts.items(), key = lambda x: x[1], reverse=True)

AttributeError: 'WordList' object has no attribute 'word_counts'